In [26]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [43]:
def get_title(soup):
    try:
        title = soup.find("span", attrs = {"class" : "a-size-large product-title-word-break"})
    
        title_value = title.text
    
        title_string = title_value.strip()

    except:
        return ""

    return title_string

def get_price(soup):
    try:
        price = soup.find("span", attrs = {"class" : "a-price-whole"})
        price_value = price.text.strip()

    except:
        return ""

    return price_value

def get_rating(soup):
    try:
        rating = soup.find("span", attrs = {"id" : "acrCustomerReviewText"})
        rating_value = rating.text.strip()
    except:
        return ""
    return rating_value


if __name__ == '__main__':
    URL = "https://www.amazon.in/s?k=speakers&crid=1Y3ALLCZHJPUV&sprefix=speakers%2Caps%2C236&ref=nb_sb_noss_1"
    HEADERS = ({'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36", "Accept-Language" : "en-US"})
    web_page = requests.get(URL, headers = HEADERS)
    soup = BeautifulSoup(web_page.content, "html.parser")
    links = soup.find_all("a", attrs={"class" : "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})

    links_list = []

    i = 0
    for link in links:
        if i <= 3:
            links_list.append(link.get("href"))
            i+=1

    d = {"title":[], "price" : [], "rating" : []}
    for link in links_list:
        new_web_page = requests.get("https://www.amazon.in"+link, headers = HEADERS)
        new_soup = BeautifulSoup(new_web_page.content, "html.parser")
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)
        

In [44]:
amazon_df

,title,price,rating
0,ZEBRONICS Juke BAR 2550 35W Bluetooth Soundbar...,"2,499.","19,142 ratings"
1,PTron Newly Launched Fusion One 14W Bluetooth ...,"1,299.",178 ratings
2,JBL Flip 5 Wireless Portable Bluetooth Speaker...,"6,999.","31,323 ratings"
3,boAt Stone 180 5W Bluetooth Speaker with Upto ...,"1,099.","20,333 ratings"
